In [1]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [5]:
def sin(t,A,F,O): return O + A * np.sin(2*np.pi*F*t)

def fft(t,y,max_freq=None): 
    x_fft   = np.fft.rfftfreq(len(t), t[1]-t[0])
    if max_freq: x_fft = x_fft[x_fft <=max_freq]
        
    y_fft = 2 * np.abs(np.fft.rfft(y)) / len(y)
    y_fft[0] /= 2
    return x_fft, y_fft[:len(x_fft)]

def add_signal(t1, y1, t2_start, t2_end, y2_gen):
    y1     = y1.copy()
    t      = (t1 >= t2_start) & (t1 <= t2_end)
    y1[t] += y2_gen(t1[t])
    return y1



    
SAMPLING_RATE = 5000   
DURATION      = 1
FFT_MAX_FREQ  = 30

t = np.arange(0,DURATION, 1/SAMPLING_RATE)



# Signal ohne Störung (blau)
A=1; F=10; O=0.5

y_normal               = sin(t, A, F, O)
x_fft, y_fft_normal    = fft(t,y_normal,FFT_MAX_FREQ)



# Signal mit Verschiebung (rot)
T_OFFSET               = 1/4 * 1/F

y_shift                = sin(t + T_OFFSET, A, F, O)
x_fft, y_fft_shift     = fft(t,y_shift,FFT_MAX_FREQ)



# Signal mit Störung (gruen)
STOERUNG_START=0.3; STOERUNG_END=.7
A_STOERUNG=0.1; F_STOERUNG=20; O_STOERUNG=0

stoerung_gen           = lambda t: sin(t, A=.1, F=20, O=0)
y_stoerung             = add_signal(t, y_shift, STOERUNG_START, STOERUNG_END, stoerung_gen)
x_fft, y_fft_stoerung  = fft(t,y_stoerung,FFT_MAX_FREQ)


fig_time = go.Figure(layout={"title" : "Zeitbereich des Signals"})
fig_time.add_scatter(x=t, y=y_normal,   name="Original-Signal")
fig_time.add_scatter(x=t, y=y_shift,    name="Original-Signal (zeitversetzt)")
fig_time.add_scatter(x=t, y=y_stoerung, name="Original-Signal (zeitversetzt) + Störung")
fig_time.show()


fig_freq = go.Figure(layout={"title" : "Frequenzbereich des Signals"})
fig_freq.add_bar(x=x_fft, y=y_fft_normal,    name="Original-Signal")
fig_freq.add_bar(x=x_fft, y=y_fft_shift,     name="Original-Signal (zeitversetzt)")
fig_freq.add_bar(x=x_fft, y=y_fft_stoerung,  name="Original-Signal (zeitversetzt) + Störung")

fig_freq.show()